In [1]:
import numpy as np
import pandas as pd

In [5]:
df = pd.DataFrame({
    'Sex': ['M', 'F', 'F', 'D', '?'],
    'Age': [29, 30, 24, 290, 25]
})
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25


The previous DataFrame doesnot have any missing values. 290 doesn't seem like a valid age, and D and ? don't correspond with any known sex category.

##Finding Unique Values

For a "categorical" type of field (like Sex, which only takes values of a discrete set ('M', 'F')), we start by analyzing the variety of values present. For that, we use the unique() method:

In [6]:
df['Sex'].unique()

array(['M', 'F', 'D', '?'], dtype=object)

In [7]:
df['Sex'].value_counts()

F    2
M    1
D    1
?    1
Name: Sex, dtype: int64

In [8]:
# for 1 row in 1 column
df['Sex'].replace('D','F')

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

In [11]:
# for more than 1 rows in 1 column
df['Sex'].replace({'D':'F', 'N':'M'})

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

If you have many columns to replace, you could apply it at "DataFrame level":

In [12]:
df.replace({
    'Sex': {
        'D':'F',
        'N':'M'
    },
    'Age': {
        290:29
    }
})

,Sex,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,?,25


In the previous example, 290 is explicitly replaced with 29 (assuming it was just an extra 0 entered at data-entry phase). But what if you'd like to remove all the extra 0s from the ages columns? (example, 150 > 15, 490 > 49).

The first step would be to just set the limit of the "not possible" age. Let's say that anything above 100 isn't credible for our dataset. We can then combine boolean selection with the operation:

In [13]:
df[df['Age']>100]

,Sex,Age
3,D,290


In [15]:
df.loc[df['Age']>100, 'Age']= df.loc[df['Age']>100, 'Age']/10
df

,Sex,Age
0,M,29.0
1,F,30.0
2,F,24.0
3,D,29.0
4,?,25.0


##Duplicates in Series

Checking duplicate values is extremely simple. It'll behave differently between Series and DataFrames. Let's start with Series. As an example, let's say we're throwing a fancy party and we're inviting Ambassadors from Europe. But can only invite one ambassador per country. This is our original list, and as you can see, both the UK and Germany have duplicated ambassadors:

In [16]:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'GÃƒÂ©rard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])


In [17]:
ambassadors

GÃƒÂ©rard Araud               France
Kim Darroch           United Kingdom
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
Peter Ammon                  Germany
Klaus Scharioth              Germany
dtype: object

* duplicated (that will tell you which values 
are duplicates)
* drop_duplicates (which will just get rid of duplicates)

In [18]:
ambassadors.duplicated()

GÃƒÂ©rard Araud       False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

By default, it'll consider the first occurrence of the value as not-duplicate. You can change this behavior with the keep parameter.

In [19]:
ambassadors.duplicated(keep='last')

GÃƒÂ©rard Araud       False
Kim Darroch            True
Peter Westmacott      False
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth       False
dtype: bool

We can also choose to mark all of them as duplicates with keep=False.

In [20]:
ambassadors.duplicated(keep=False)

GÃƒÂ©rard Araud       False
Kim Darroch            True
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In [21]:
ambassadors.drop_duplicates()

GÃƒÂ©rard Araud               France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

In [22]:
ambassadors.drop_duplicates(keep='last')

GÃƒÂ©rard Araud               France
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

In [25]:
ambassadors.drop_duplicates(keep=False)

GÃƒÂ©rard Araud       France
Armando Varricchio     Italy
dtype: object

##Duplicates in DataFames

Two rows with exactly the same values are considered to be duplicates.

In [26]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})

In [27]:
players

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [28]:
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

Conceptually, "duplicated" means "all the column values should be duplicates". We can customize this with the subset parameter.

In [29]:
players.duplicated(subset=['Name'])

0    False
1    False
2     True
3    False
4     True
dtype: bool

In [30]:
players.duplicated(subset=['Name'], keep='last')

0     True
1    False
2     True
3    False
4    False
dtype: bool

In [31]:
players.drop_duplicates()

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [32]:
players.drop_duplicates(subset=['Name'])

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
3,Carmelo Anthony,SF


In [33]:
players.drop_duplicates(subset=['Name'], keep='last')

,Name,Pos
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


##Text Handling

##Splitting columns

In [50]:
df = pd.DataFrame({
    'Data': [
        '1987_M_US _1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
]})

The single columns represent the values "year, Sex, Country and number of children", but it's all been grouped in the same column and separated by an underscore.

In [51]:
df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


In [52]:
df['Data'].str.split('_')

0       [1987, M, US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [53]:
df= df['Data'].str.split('_', expand=True)

In [54]:
df

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [55]:
df.columns = ['Year', 'Sex', 'Country', 'No Children']

In [56]:
df

,Year,Sex,Country,No Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


`contains` method checks which columns contain a given value. It takes a regex/pattern as first value, so we need to escape the ? symbol as it has a special meaning for these patterns. Regular letters don't need escaping.

In [57]:
df['Year'].str.contains('\?')

0    False
1     True
2    False
3     True
4    False
Name: Year, dtype: bool

In [58]:
df['Year'].str.contains('U')

0    False
1    False
2    False
3    False
4    False
Name: Year, dtype: bool

Removing blank spaces can be achieved with `strip` (`lstrip` and `rstrip` also exist) or just `replace`.

In [59]:
df['Country'].str.strip()

0      US
1      UK
2      US
3      IT
4    I  T
Name: Country, dtype: object

In [61]:
df['Country'].str.replace(' ','')

0    US
1    UK
2    US
3    IT
4    IT
Name: Country, dtype: object

`replace` and `contains` take regex patterns, which can make it easier to replace values in bulk.